In [83]:
import librosa
import IPython.display as ipd
%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
import sklearn as sk
import pandas as pd
from sklearn import preprocessing
import wav_to_mel_spect
import os
import pandas as pd
from scipy.io import wavfile

In [146]:
PATHSAMPLES = '../data/training_list.txt'
NSAMPLES = 500
SOUNDS =np.array(['yes','no','right','five','nine'])
path_out = '/home/deiry/semilleroML/Voice_Recognition/data/mel-spectrograms/'

In [110]:
def get_Path_Word_SpeakerId(NSAMPLES, PATHSAMPLES, SOUNDS):
    ROOTPATH= 'data/'
    data = pd.read_csv(PATHSAMPLES, sep=" ", header=None)
    data = data.values #Convertir en numpy array
    labels = []
    speakers  = []
    paths = []
    for path in data:
        pathSplit =path[0].split('/')
        speakerName = pathSplit[1].split('_')[0]
        word = pathSplit[0]
        isWord = np.where(SOUNDS == word)
        if len(isWord[0]) != 0:
            label = SOUNDS[isWord[0][0]]
            labels.append(label)
            speakers.append(speakerName)
            paths.append(ROOTPATH+path[0])
    NSAMPLES = NSAMPLES
    first = labels.index(SOUNDS[0])
    last = first +NSAMPLES
    matrixMajor = np.column_stack([paths[first:last], labels[first:last], speakers[first:last]])    
    for i in range(1,np.size(SOUNDS,0)):
        first = labels.index(SOUNDS[i])
        last = first +NSAMPLES
        clasei = np.column_stack([paths[first:last], labels[first:last], speakers[first:last]])    
        matrixMajor = np.append(matrixMajor,clasei,axis=0)
    return matrixMajor

In [129]:
def createDataset(path_word_speakerId):
    speakers =[]
    words = []
    spectrograms = []

     #Encode etiqueta del path audio,speaker y la palabra
    leSpeaker = preprocessing.LabelEncoder()
    leWord =  preprocessing.LabelEncoder()

    #Encode
    leSpeaker.fit(path_word_speakerId[:,2])
    leWord.fit(path_word_speakerId[:,1])


    for p in path_word_speakerId:
        name = p[0].split('/')[-1].split('.')[0]+'.jpg'
        word = p[1]
        speaker_id = p[-1]

        #Transformación del encoder
        labelSpeaker = leSpeaker.transform([speaker_id])
        labelWord= leWord.transform([word])     
        wav_path = '../'+p[0]
        spect = wavToMelSpect(wav_path) 
        spectrograms.append(spect)
        speakers.append(labelSpeaker[0])
        words.append(labelWord[0])
    return spectrograms,speakers, words

In [147]:
path_word_speakerId = get_Path_Word_SpeakerId(NSAMPLES, PATHSAMPLES, SOUNDS)

In [148]:
path_word_speakerId.shape

(2500, 3)

In [149]:
spectrograms,speakers, words = createDataset(path_word_speakerId)

In [150]:
df = pd.DataFrame([spectrograms,speakers,words],index=['mel-spectrogram','speaker','word']).T
df

,mel-spectrogram,speaker,word
0,"[[864.48065, 868.26074, 866.4554, 870.45496, 8...",0,4
1,"[[0.0, 0.0, 65.97594, 885.9732, 1072.2451, 843...",0,4
2,"[[1061.283, 1060.43, 1060.2893, 1062.0586, 106...",1,4
3,"[[927.7823, 934.6374, 930.5045, 934.30133, 933...",3,4
4,"[[852.81067, 854.01556, 853.04205, 853.4441, 8...",3,4
...,...,...,...
2495,"[[0.0, 0.0, 103.94041, 1383.8752, 1667.4603, 1...",297,1
2496,"[[1543.1268, 1540.9369, 1534.9974, 1541.4904, ...",297,1
2497,"[[1475.6971, 1474.5621, 1474.6218, 1474.463, 1...",297,1
2498,"[[391.0237, 395.27707, 394.7014, 394.7173, 394...",298,1


In [151]:
df

,mel-spectrogram,speaker,word
0,"[[864.48065, 868.26074, 866.4554, 870.45496, 8...",0,4
1,"[[0.0, 0.0, 65.97594, 885.9732, 1072.2451, 843...",0,4
2,"[[1061.283, 1060.43, 1060.2893, 1062.0586, 106...",1,4
3,"[[927.7823, 934.6374, 930.5045, 934.30133, 933...",3,4
4,"[[852.81067, 854.01556, 853.04205, 853.4441, 8...",3,4
...,...,...,...
2495,"[[0.0, 0.0, 103.94041, 1383.8752, 1667.4603, 1...",297,1
2496,"[[1543.1268, 1540.9369, 1534.9974, 1541.4904, ...",297,1
2497,"[[1475.6971, 1474.5621, 1474.6218, 1474.463, 1...",297,1
2498,"[[391.0237, 395.27707, 394.7014, 394.7173, 394...",298,1


In [139]:
pwd

'/home/deiry/semilleroML/Voice_Recognition/features'

In [152]:
df.to_hdf('/home/deiry/semilleroML/Voice_Recognition/data/mel-spectrograms/mel-spectrograms_2500.hd5', key = 'description')

/home/deiry/semilleroML/semillero/lib/python3.7/site-packages/pandas/core/generic.py:2530: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->['mel-spectrogram', 'speaker', 'word']]

  pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [153]:
df_b = pd.read_hdf('/home/deiry/semilleroML/Voice_Recognition/data/mel-spectrograms/mel-spectrograms_2500.hd5')

In [154]:
df_b.shape

(2500, 3)